# Organising the Nasdaq dataset

**File**:    01_importing_crypto_time_series.ipnv

**Purpose**: Selecting time periodes of the Nasdaq dataset to decide different trading behaviours.

**Authors**: Gabriel R. Palma, Mariusz Skoczeń and Phil Maguire

**Date**:    09/01/24

# Modules used in this project

In [3]:
import pandas as pd
import requests
import json
import numpy as np
import time
import os

# Functions used in this project

In [4]:
PATH = '../output_data/'

def get_candles_okx(symbol='BTC-USDT-SWAP', interval='1h', limit=100, start_time=None, end_time=None):
    data = {'instId': symbol, 'bar': interval, 'limit': limit}
    if start_time is not None:
        data['after'] = str(start_time)
    if end_time is not None:
        data['before'] = str(end_time)
    response = requests.get('https://www.okx.com/api/v5/market/history-candles', params=data)
    resp = response.text
    ret = [[float(y) for y in x] for x in json.loads(resp)['data']]
    return ret


def get_okx_data(symbol='BTC-USDT-SWAP', save_path='', limit=100, sleep=0.1, interval='1m', row_limit=1e7):
    price = []
    while True:
        end = None if len(price) == 0 else int(price[-1][0])
        print('Downloading', symbol, end)
        response = []
        error_count = 0
        while True:
            try:
                response = get_candles_okx(symbol=symbol, interval=interval, limit=limit,
                                           start_time=end, end_time=None)
                break
            except:
                error_count += 1
                time.sleep(sleep)
                print('Error detected downloading', symbol, end)
                if error_count >= 5:
                    break
        if len(response) == 0 or (len(price) > 0 and response[0][0] > price[-1][0]):
            break
        price = price + response
        if len(price) > row_limit:
            break
        time.sleep(sleep)
    arr = np.asarray(price, dtype=np.float64)
    np.savetxt(PATH + save_path + symbol + '.csv', arr, delimiter=",")

# Obtaining the crypto time series from OKX

In [2]:
#cryptocurrencies = ['BTC', 'ETH', 'USDT', 'BNB', 'USDC', 'XRP', 'ADA', 'MATIC', 'DOGE', 'BUSD', 'SOL', 'DOT', 'SHIB', 'DAI', 'TRX', 'AVAX', 'WBTC', 'LEO', 'LTC', 'ATOM', 'XMR', 'OKB', 'LINK', 'ETC', 'XLM', 'CRO', 'BCH', 'ALGO', 'QNT', 'VET', 'ICP', 'FIL', 'HBAR', 'FLOW', 'NEAR', 'USDP', 'EGLD', 'TUSD', 'APE', 'MANA', 'SAND', 'THETA', 'XTZ', 'AXS', 'EOS', 'AAVE', 'USDD', 'HT', 'BSV', 'KCS']
cryptocurrencies = ['BTC', 'ETH', 'BNB', 'USDC', 'XRP', 'ADA', 'MATIC', 'DOGE', 'SOL', 'DOT', 'SHIB', 'DAI', 'TRX', 'AVAX', 'WBTC', 'LEO', 'LTC', 'ATOM', 'OKB', 'LINK', 'ETC', 'XLM', 'CRO', 'BCH', 'ALGO', 'ICP', 'FIL', 'HBAR', 'FLOW', 'NEAR', 'EGLD', 'APE', 'MANA', 'SAND', 'THETA', 'XTZ', 'AXS', 'EOS', 'AAVE', 'BSV']

In [24]:
for coin in cryptocurrencies:
    get_okx_data(symbol='%s-USDT'%coin, interval='1m', sleep=0, row_limit=60*60*1*1*1)